## Part 1: 搭建一个分词工具

### Part 1.1  基于枚举方法来搭建中文分词工具

此项目需要的数据：
1. 综合类中文词库.xlsx： 包含了中文词，当做词典来用
2. 以变量的方式提供了部分unigram概率 word_prob


举个例子： 给定词典=[我们 学习 人工 智能 人工智能 未来 是]， 另外我们给定unigram概率：p(我们)=0.25, p(学习)=0.15, p(人工)=0.05, p(智能)=0.1, p(人工智能)=0.2, p(未来)=0.1, p(是)=0.15

#### Step 1: 对于给定字符串：”我们学习人工智能，人工智能是未来“, 找出所有可能的分割方式
- [我们，学习，人工智能，人工智能，是，未来]
- [我们，学习，人工，智能，人工智能，是，未来]
- [我们，学习，人工，智能，人工，智能，是，未来]
- [我们，学习，人工智能，人工，智能，是，未来]
.......


#### Step 2: 我们也可以计算出每一个切分之后句子的概率
- p(我们，学习，人工智能，人工智能，是，未来)= -log p(我们)-log p(学习)-log p(人工智能)-log p(人工智能)-log p(是)-log p(未来)
- p(我们，学习，人工，智能，人工智能，是，未来)=-log p(我们)-log p(学习)-log p(人工)-log p(智能)-log p(人工智能)-log p(是)-log p(未来)
- p(我们，学习，人工，智能，人工，智能，是，未来)=-log p(我们)-log p(学习)-log p(人工)-log p(智能)-log p(人工)-log p(智能)-log p(是)-log p(未来)
- p(我们，学习，人工智能，人工，智能，是，未来)=-log p(我们)-log p(学习)-log p(人工智能)-log p(人工)-log p(智能)-log(是)-log p(未来)
.....

#### Step 3: 返回第二步中概率最大的结果

## 1.基本准备过程
- 使用pandas读取csv,作为词典
- 没有出现的词的概率设为default


In [5]:
import pandas as pd
path='doc.csv'
data=pd.read_csv(path,usecols=[0],encoding='gb18030')
data=data.to_numpy()
data=data.tolist()
dic_words ={}   # 保存词典库中读取的单词
for d in data:
    if d[0] not in dic_words:
        dic_words[d[0]]=1
    else:
        dic_words[d[0]]+=1

#字典存储概率
word_prob = {"北京":0.03,"的":0.08,"天":0.005,"气":0.005,"天气":0.06,"真":0.04,"好":0.05,"真好":0.04,"啊":0.01,"真好啊":0.02, 
             "今":0.01,"今天":0.07,"课程":0.06,"内容":0.06,"有":0.05,"很":0.03,"很有":0.04,"意思":0.06,"有意思":0.005,"课":0.01,
             "程":0.005,"经常":0.08,"意见":0.08,"意":0.01,"见":0.005,"有意见":0.02,"分歧":0.04,"分":0.02, "歧":0.005}
default_prob=0.0001


## 2.基于匹配的分词算法
- 最大向前匹配
    ：设置一个最大匹配的长度，根据这个长度从头开始匹配，如果没有这样的词，那么将长度减1，再次进行匹配，直到找到相应的词或者长度为1.再重复这个过程
- 最大向后匹配：
  同理 只是从末尾开始
- 双向最大匹配
    从最大向前匹配和最大向后匹配选择比较好的结果
- 参考博客：https://blog.csdn.net/selinda001/article/details/79345072

In [6]:
import math
def Max_forwardMatch(input_str):
    segments = []   
    sentence=input_str
    Maxlen=5
    i=0
    forword_sub_seg=[]
    while(i<len(sentence)):
        max_len=Maxlen
        while(max_len>=1):
            if sentence[i:i+max_len] in dic_words:
                break
            else:
                max_len=max_len-1
        forword_sub_seg.append(sentence[i:i+max_len])
        i=i+max_len
    segments.append(forword_sub_seg)
    return segments
def Max_backwardMatch(input_str):
    
    segments=[]
    
    sentence=input_str
    
    i=len(sentence)
   
    Maxlen=5
    i=len(sentence)
    back_sub_seg=[]
    
    max_len=Maxlen
    while(i>max_len):
        while(max_len>=1):
            if sentence[i-max_len:i] in dic_words:
                break
            else:
                max_len=max_len-1
        back_sub_seg.append(sentence[i-max_len:i])
        i=i-max_len
          #print(sentence[i])
    if i>0:
          back_sub_seg.append(sentence[:i])
    back_sub_seg.reverse()
    segments.append(back_sub_seg)
    return segments
    
def word_segment_naive(input_str):
    """
    1. 对于输入字符串做分词，并返回所有可行的分词之后的结果。
    2. 针对于每一个返回结果，计算句子的概率
    3. 返回概率最高的最作为最后结果
    """
    segments = []  
    
    sentence=input_str
  
    forward_segments=Max_forwardMatch(input_str)
    backward_segments=Max_backwardMatch(input_str)
    
    segments+=forward_segments
    segments+=backward_segments
  


    # TODO: 第二步：循环所有的分词结果，并计算出概率最高的分词结果，并返回
    best_segment = segments[0]
    best_score = 1.0
    for seg in segments:
        score=0.0
        for word in seg:
            if word in word_prob:
                score=score+(-math.log(word_prob[word]))
            else:
                score=score+(-math.log(default_prob))
        if(score<best_score):
            best_score=score
            best_segment=seg
    print("best:{}".format(best_segment))
    return best_segment      

In [7]:
# 测试
print (word_segment_naive("北京的天气真好啊"))
print (word_segment_naive("今天的课程内容很有意思"))
print (word_segment_naive("经常有意见分歧"))

best:['北京', '的', '天气', '真好', '啊']
['北京', '的', '天气', '真好', '啊']
best:['今天', '的', '课程', '内容', '很', '有意思']
['今天', '的', '课程', '内容', '很', '有意思']
best:['经常', '有意', '见', '分歧']
['经常', '有意', '见', '分歧']


### Part 1.2  基于维特比算法来优化上述流程

此项目需要的数据：
1. 综合类中文词库.xlsx： 包含了中文词，当做词典来用
2. 以变量的方式提供了部分unigram概率word_prob


举个例子： 给定词典=[我们 学习 人工 智能 人工智能 未来 是]， 另外我们给定unigram概率：p(我们)=0.25, p(学习)=0.15, p(人工)=0.05, p(智能)=0.1, p(人工智能)=0.2, p(未来)=0.1, p(是)=0.15

#### Step 1: 根据词典，输入的句子和 word_prob来创建带权重的有向图（Directed Graph）
有向图的每一条边是一个单词的概率（只要存在于词典里的都可以作为一个合法的单词），这些概率已经给出（存放在word_prob）。
使用字典来表示邻接表:两个字典，一个存储邻接节点的下标，一个存储权重。

#### Step 2: 编写维特比算法（viterebi）算法来找出其中最好的PATH， 也就是最好的句子切分
使用dp求解最优的切分。dp[i]表示以第i个词结尾的单词的句子当前最小的权重。dp_path[i]记录最优的到达节点i的节点的下标。


#### Step 3: 返回结果
通过记录的路径，倒着切割句子，最后将其反转即可。

In [8]:

def word_segment_viterbi(input_str):
    """
    1. 基于输入字符串，词典，以及给定的unigram概率来创建DAG(有向图）。
    2. 编写维特比算法来寻找最优的PATH
    3. 返回分词结果
    
    input_str: 输入字符串   输入格式：“今天天气好”
    best_segment: 最好的分词结果  输出格式：["今天"，"天气"，"好"]
    """
    
    
    #基本信息的准备
    sentence=[]
    for s in input_str:
        sentence.append(s)
    s_len=len(sentence)
    
    word2index={}
    for word,index in enumerate(sentence):#编号从1开始
        word2index[word]=index
        
    #使用两个dict表示邻接表
    #一个表示与那些词有关系
    #另一个存储那些词的权重
    dic_link={}
    dic_weight={}
  
    for i in range(s_len):
        link=[]
        weight=[]
        for j in range(i+1):
          #每个词都查看其前面的词有没有链接
            if input_str[j:i+1] in dic_words:
                link.append(j)
                prob=0.0
                if input_str[j:i+1] in word_prob:
                    prob=-math.log(word_prob[input_str[j:i+1]])
                else:
                    prob=-math.log(default_prob)
                weight.append(prob)
        dic_link[i]=link
        dic_weight[i]=weight

              
    #进行计算 最短路径 使用dp计算最短路径
    dp=[0.0]*(s_len+1)
    dp_path=[0]*(s_len+1)
    
    #检查一列的值就是可能右边的值
    for i in range(1,s_len+1):
        best_value=10000.0
        bast_path=i-1
        for j in range(len(dic_link[i-1])):#每个节点的临域
                index=dic_link[i-1][j]
                value=dp[index]+dic_weight[i-1][j]
                if value-best_value<1e-9:
                    best_value=value
                    best_path=index
        dp[i]=best_value #记录每一个节点对应点最好路径和值
        dp_path[i]=best_path           
                    
    #根据最好的PATH, 返回最好的切分
    #倒着来切割
    path_index=s_len
    path_list=[]
    while path_index>0:
        incoming_index=dp_path[path_index]
        path_list.append(input_str[incoming_index:path_index])
        path_index=incoming_index
    path_list.reverse()
    best_segment=path_list
    
    return best_segment      

In [9]:
# 测试
print (word_segment_viterbi("北京的天气真好啊"))
print (word_segment_viterbi("今天的课程内容很有意思"))
print (word_segment_viterbi("经常有意见分歧"))

['北京', '的', '天气', '真好', '啊']
['今天', '的', '课程', '内容', '很', '有意思']
['经常', '有', '意见', '分歧']


## Part 2:  搭建一个简单的问答系统

本次项目的目标是搭建一个基于检索式的简单的问答系统。

通过此项目，你将会有机会掌握以下几个知识点：
1. 字符串操作   2. 文本预处理技术（词过滤，标准化）   3. 文本的表示（tf-idf, word2vec)  4. 文本相似度计算  5. 文本高效检索

此项目需要的数据：
1. train-v2.0.json: 这个数据包含了问题和答案的pair， 但是以JSON格式存在，需要编写parser来提取出里面的问题和答案。 
2. glove.6B: 这个文件需要从网上下载，下载地址为：https://nlp.stanford.edu/projects/glove/， 请使用d=100的词向量

### Part 2.1  第一部分： 读取文件，并把内容分别写到两个list里（一个list对应问题集，另一个list对应答案集）
首先需要分析数据集的形式，然后读出question,answer

In [22]:

import json
def read_corpus():
    path='train-v2.0.json'
    f=open(path,encoding='utf-8')
    dic=json.load(f)
    data=dic["data"][0]["paragraphs"]
    qlist=[]
    alist=[]
    for i in range(len(data)):
        data_qas=data[i]["qas"]
        for j in range(len(data_qas)):
            qlist.append(data_qas[j]["question"])
            answer=data_qas[j]["answers"][0]["text"]
            alist.append(answer)
    assert len(qlist) == len(alist)  # 确保长度一样
    #print(qlist)
    return qlist, alist


### Part 2.2 理解数据（可视化分析/统计信息）
对数据的理解是任何AI工作的第一步，需要充分对手上的数据有个更直观的理解。 

In [23]:

def word2dic(wordlist):
    #只考虑qlist
    word2freq={}
    for sentence in wordlist:
        #print(sentence)
        sentence=sentence.split()
        for word in sentence:
            if word in word2freq:
                word2freq[word]+=1
            else:
                word2freq[word]=1    
    #将词典按照频次排序
    sorted(word2freq.values(),reverse=True)
    word2index={}
    index2word={}
    words_total=0
    index=0
    for word in word2freq:
        word2index[word]=index
        index2word[index]=word
        words_total+=word2freq[word]
        index+=1
        
    return word2freq,word2index,index2word,words_total  
    
        

经过可视化分析，我们可以看到词表的频率分布，满足*齐夫定律*：词的频率与其频率的排名呈反比。

In [24]:
import matplotlib.pyplot as plt
import numpy as np
def Visualization(wordlist):
    word2freq,word2index,_,_=word2dic(wordlist)
    word_index=[]
    word_freq=[]
    for word in word2freq:
        word_index.append(word2index[word])
        word_freq.append(word2freq[word])
        
    #转换成np
    np.array(word_index)
    np.array(word_freq)
    
    plt.title("WOrd Frequency")
    plt.plot(word_index,word_freq)
    plt.show()

### 2.3 文本预处理
- 使用正则表达式去掉特殊符号
- 使用NLTK的停用词表取出停用词
- 使用split进行英文分词，使用lower统一大小写
- 使用NLTK词干工具 进行词干抽取
- 过滤低频词 设定阈值 比如5

In [25]:

import nltk
from nltk.stem import WordNetLemmatizer
import re 
def Preprocessing(wordlist):
    
    nltk.download('stopwords')
    nltk.download('wordnet')
    stopword_list=nltk.corpus.stopwords.words('english')
    
    #step1:去除字符串的特殊符号 
    special_character_removal = re.compile(r'[^a-z\d ]', re.IGNORECASE)
    for i in range(len(wordlist)):
        wordlist[i]=special_character_removal.sub('',wordlist[i])
    
    #step2.取出停用词
    for i in range(len(wordlist)):
        tmp=[]
        wordlist[i]=wordlist[i].split()
        for w in wordlist[i]:
            w=w.lower()
            if w not in stopword_list:
                tmp.append(w)
        wordlist[i]=tmp
        
    
    # step3:统计词频 过滤低频词
    worddic={}
    for q in wordlist:
        for w in q:
            if w not in worddic:
                worddic[w]=1
            else:
                worddic[w]+=1
                
    for i in range(len(wordlist)):
        tmp_q=[]
        for w in wordlist[i]:
            if worddic[w]>=1:
                tmp_q.append(w)
        wordlist[i]=tmp_q
    
    wordnet_Lemmatizer=WordNetLemmatizer()
    for i in range(len(wordlist)):
        for j in range(len(wordlist[i])):
            wordlist[i][j]=wordnet_Lemmatizer.lemmatize(wordlist[i][j])
            
    return wordlist
    

### 2.4 文本表示
- 将向量转化为tf_idf向量
- 向量大小为词表长度
- 分别求每一个词的tf和idf

In [26]:

import math
def Vectorization(wordlist,x):
    word2freq,word2index,_,word_total=word2dic(wordlist)
    wordlist=Preprocessing(wordlist)
    x=Preprocessing(x)
    word2tdf={}
    for word in word2freq:
        tf=word2freq[word]*1.0/word_total
        idf=0
        for sentence in wordlist:
            if word in sentence:
                idf+=1
        idf=math.log((1.0+1.0)/(idf+1.0))
        tf_idf=tf*idf
        word2tdf[word]=tf_idf
        
    X=[]
    for sentence in wordlist:
        vec=[0]*word_total
        for word in sentence:
            if word in word2index and word in word2tdf:
                vec[word2index[word]]=word2tdf[word]
        X.append(vec)
        
    X_test=[]
    for sencente in x:
        vec=[0]*word_total
        for word in sentence:
            if word in word2index and word in word2tdf:
                vec[word2index[word]]=word2tdf[word]
        X_test.append(vec)
            
    
    return X,X_test
            
        
  

### 2.5 对于用户的输入问题，找到相似度最高的TOP5问题，并把5个潜在的答案做返回

In [27]:

def top5results(input_q):
    """
    给定用户输入的问题 input_q, 返回最有可能的TOP 5问题。这里面需要做到以下几点：
    1. 对于用户的输入 input_q 首先做一系列的预处理，然后再转换成tf-idf向量（利用上面的vectorizer)
    2. 计算跟每个库里的问题之间的相似度
    3. 找出相似度最高的top5问题的答案
    """
   
    qlist,alist=read_corpus()
  

    input_x=[]
    input_x.append(input_q)
    
    X,X_test=Vectorization(qlist,input_x)
    
    X=np.array(X)

    X_test=np.array(X_test)

    index2sim={}
    #使用点积求解相似度
    
    for i in range(X.shape[0]):
        similarity=np.dot(X_test[0],X[i])
        index2sim[i]=similarity
        
    sorted(index2sim.values(),reverse=True)

    top_index=[]
    for i in index2sim:
        top_index.append(i)
    
    answer=[]
    for i in top_index:
        answer.append(alist[i])
    return answer[:5]  # 返回相似度最高的问题对应的答案，作为TOP5答案    

### 2.6 利用倒排表的优化。 
上面的算法，一个最大的缺点是每一个用户问题都需要跟库里的所有的问题都计算相似度。假设我们库里的问题非常多，这将是效率非常低的方法。 这里面一个方案是通过倒排表的方式，先从库里面找到跟当前的输入类似的问题描述。然后针对于这些candidates问题再做余弦相似度的计算。这样会节省大量的时间。
- 倒排表的思路是层层过滤
- 也就是不是用所有的问题作为比对集合，而是筛选出候选。对于输入的问题的每一个词，看看他出现在那些句子中，将这些句子放在一起，也就是候选集合。

In [28]:

def getInvertedTable(input_q,qlist):
    word2doc={}
    input_q=input_q.split()
    for word in input_q:
        doc=[]
        for i in range(len(qlist)):
            if word in qlist[i]:
                doc.append(i)
        word2doc[word]=doc
    return word2doc
                  
def top5results_invidx(input_q):
    """
    给定用户输入的问题 input_q, 返回最有可能的TOP 5问题。这里面需要做到以下几点：
    1. 利用倒排表来筛选 candidate
    2. 对于用户的输入 input_q 首先做一系列的预处理，然后再转换成tf-idf向量（利用上面的vectorizer)
    3. 计算跟每个库里的问题之间的相似度
    4. 找出相似度最高的top5问题的答案
    """
    qlist,alist=read_corpus()
    #1.建立到排表
    word2doc=getInvertedTable(input_q,qlist)
    input_x=[]
    input_x.append(input_q)
    X,X_test=Vectorization(qlist,input_x)
    
    #合并候选doc
    doc_candidate=[]
    for word in word2doc:
        doc_candidate+=word2doc[word]
    doc_set=list(set(doc_candidate))
    
    doc2freq={}
    
    #对doc按照频率计数
    num=0
    for doc in doc_set:
        for d in doc_candidate:
            if doc==d:
                num+=1
        doc2freq[doc]=num
    
    sorted(doc2freq.values(),reverse=True)
    
    #候选doc的数量 选取前80%
    doc_num=len(doc2freq)
    
    doc_select=int(doc_num*0.8)
    
    X_candidate=[]
    i=0
    for doc in doc2freq:
        X_candidate.append(X[doc])
        i+=1
        if i>doc_select:
            break
            
    X_candidate=np.array(X_candidate)
    X_test=np.array(X_test)
    index2sim={}
    
    #和候选列表进行计算相似度
    for i in range(X_candidate.shape[0]):
        sim=np.dot(X_test[0],X_candidate[i])
        index2sim[doc_set[i]]=sim
       
    sorted(index2sim.values(),reverse=True)
    answer=[]
    
    for index in index2sim:
        answer.append(alist[index])
        
    return answer[:10]
        

### 2.7 基于词向量的文本表示
上面所用到的方法论是基于词袋模型（bag-of-words model）。这样的方法论有两个主要的问题：1. 无法计算词语之间的相似度  2. 稀疏度很高。 在2.7里面我们
讲采用词向量作为文本的表示。词向量方面需要下载： https://nlp.stanford.edu/projects/glove/ （请下载glove.6B.zip），并使用d=100的词向量（100维）。


In [51]:

#emb = # 读取每一个单词的嵌入。这个是 D*H的矩阵，这里的D是词典库的大小， H是词向量的大小。 这里面我们给定的每个单词的词向量，那句子向量怎么表达？
      # 其中，最简单的方式 句子向量 = 词向量的平均（出现在问句里的）， 如果给定的词没有出现在词典库里，则忽略掉这个词。
    
def read_glove():
    glove_path='glove.6B.100d.txt'
    with open(glove_path,encoding='utf-8') as f:
        wordset=set()
        word2vec={}
        
        for line in f:
            line=line.strip().split()
            wordset.add(line[0])
            word2vec[line[0]]=np.array([line[1:]],dtype=float)
    return word2vec

def glove_vectorization(word2vec,input_str):
    sentence=input_str.split()
    X=np.zeros((1,100))
    for word in sentence:
        if word in word2vec:
            X+=word2vec[word]
    X=X*1.0/(len(sentence))
    return X
    
def top5results_emb(input_q):
    """
    给定用户输入的问题 input_q, 返回最有可能的TOP 5问题。这里面需要做到以下几点：
    1. 利用倒排表来筛选 candidate
    2. 对于用户的输入 input_q，转换成句子向量
    3. 计算跟每个库里的问题之间的相似度
    4. 找出相似度最高的top5问题的答案
    """
    qlist,alist=read_corpus()
    #1.建立到排表
    word2doc=getInvertedTable(input_q,qlist)
    
    doc_candidate=[]
    for word in word2doc:
        doc_candidate+=word2doc[word]
    doc_set=list(set(doc_candidate))
    
    #使用平均向量来表示 这样就不用考虑句子长度了
    word2vec=read_glove()
    
    index2sim={}
    X_test=glove_vectorization(word2vec,input_q).transpose(1,0)
    for doc in doc_set:
        input_str=qlist[doc]
        X_q=glove_vectorization(word2vec,input_str)
        sim=np.dot(X_q,X_test)
        index2sim[doc]=sim
    sorted(index2sim.values(),reverse=True)
    
    answer=[]
    for index in index2sim:
        answer.append(alist[index])
        
    return answer[:5]
    
    
    
    
